# EPL 23/24 Commentary Generator:Gemma 2b
------------------------------------------

This project aims to develop a multimodal video captioning model using Gemma2b to describe video content. By analyzing videos, we will extract salient information and generate natural language descriptions. Our ultimate goal is to generate sports commentary from videos, analyzing player techniques, game situations, and more.

In [1]:
# install required libraries
!pip install accelerate
!pip install transformers
!pip install bitsandbytes
!pip install jq
!pip install sentence_transformers langchain langchain-community chromadb

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 122.4/122.4 MB 3.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 737.4/737.4 kB 10.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 67.3/67.3 kB 2.5 MB/s eta 0:00:00
  Installing build dependencies ... - \ | / done
  Getting requirements to build wheel ... - done
  Preparing metadata (pyproject.toml) ... - done
  Preparing metadata (setup.py) ... - \ done
INFO: pip is looking at multiple versions of opentelemetry-sdk to determine which version is compatible with other requirements. This could take a while.
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 245.3/245.3 kB 8.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 MB 23.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.4/2.4 MB 56.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 604.0/604.0 kB 32.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.4/2.4 MB 52.8 MB/s eta 0:00:00
   ━━

In [2]:
from transformers import AutoTokenizer, AutoModelForCausalLM

from langchain.document_loaders import CSVLoader, JSONLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.embeddings import HuggingFaceEmbeddings
from langchain.vectorstores import Chroma

from IPython.display import display, Markdown

import torch
import pandas as pd

import json
from pathlib import Path
from pprint import pprint

In [3]:
DEVICE = torch.device("cuda") if torch.cuda.is_available() else torch.device("cpu")
MODEL_PATH = "/kaggle/input/gemma/transformers/2b-it/3"
RAG_DATA_TYPE = 'json'

if RAG_DATA_TYPE == 'csv':
    RAG_PATH = "/kaggle/input/sport-rag-datafor-sport-commentary/sport_rag_data.csv"
    NUM_RETRIEVED_DOCS = 5
    df = pd.read_csv(RAG_PATH)
    display(df.head(5))
else:
    RAG_PATH = "/kaggle/input/sport-rag-datafor-sport-commentary/sport_rag_data.json"
    NUM_RETRIEVED_DOCS = 3
    data = json.loads(Path(RAG_PATH).read_text())
    display(pprint(data))

{'documents': [{'content': 'Table tennis is derived from lawn tennis and was '
                           'initially played as after-dinner entertainment '
                           'among upper-class English families who would use '
                           'whatever they could find as equipment. At the '
                           'time, they would place books on a table to serve '
                           'as a net, the lids of cigar boxes for rackets and '
                           'a rounded-off cork from a champagne bottle as the '
                           'ball. Thankfully, table tennis equipment has '
                           'evolved over time and in 1926 competitions were '
                           'organised in Berlin and London, giving rise to the '
                           'first world championships which were held in '
                           'London that same year. Enormously popular in Asia, '
                           'table tennis is played by over 4

None

## Load Data

- Athelete
    - name
    - nation
    - gender
    - clothes color
    - world ranking
- Game
    - sports
    - competition
    - round
    - date
- Previous News
    - title
    - url
- Video Captioning
    - llava-caption

## Video Captioning
- We got video captioning result from Video-Llava
- Video-Llava gets a short video from user and deliver the result to Gemma

In [4]:
# table tennis
VIDEO_CAPTION_1 = "The video shows a ping pong match between two women playing on a table tennis court. One of the players is wearing a yellow shirt, while the other is wearing a blue shirt. The players are seen hitting the ball back and forth over the net, with the ball landing on the table. The players are actively engaged in the game, displaying their skills and techniques. The video captures the intensity and excitement of the match, showcasing the players' competitive spirit and the fast-paced nature of the sport."

# artistic gymnastics
VIDEO_CAPTION_2 = "The video features a female gymnast performing a routine on a balance beam. She starts by jumping onto the beam and executing a series of flips and turns, showcasing her impressive athleticism and balance. The gymnast then jumps off the beam and lands on the mat with her arms raised in the air, indicating a successful performance. The crowd cheers and applauds her as she completes the routine."

# diving
VIDEO_CAPTION_3 = "In the video, a man is seen diving into a pool from a diving board, and he is captured in mid-air as he jumps off the board. The man's body is seen arching as he descends into the water, creating a splash. The dive is executed with precision and skill, showcasing the diver's athleticism and control. The video captures the moment of impact as the man hits the water, creating a visually striking scene. The dive is a testament to the diver's abilities and the beauty of the sport."

## AI Agent

In [5]:
class AIAgent:
    """
    Gemma 2b-it assistant.
    It uses Gemma transformers 2b-it/3.
    """
    def __init__(self, model_path, max_length=1000):
        self.max_length = max_length
        self.tokenizer = AutoTokenizer.from_pretrained(model_path)
        self.gemma_lm = AutoModelForCausalLM.from_pretrained(model_path,device_map="auto")

    def create_prompt(self, query, video_caption, context):
        # prompt template
        prompt = f"""
        You are an AI agent specialized in creating commentary script of live sports events.
        Describe the game using the video caption provided (Video Caption).
        Describe the sport and the player using the context provided (Context).
        In order to create the commentary, please use the information from the context provided (Context).
        If needed, include also explanations.
        Video Caption: {video_caption}
        Question: {query}
        Context: {context}
        Answer:
        """
        return prompt
    
    def generate(self, query, video_caption, retrieved_info):
        prompt = self.create_prompt(query, video_caption, retrieved_info)
        input_ids = self.tokenizer(prompt, return_tensors="pt").to(DEVICE).input_ids
        # Answer generation
        answer = self.gemma_lm.generate(
            input_ids,
            #max_length=self.max_length, # limit the answer to max_length
            max_new_tokens=self.max_length
        )
        # Decode and return the answer
        answer = self.tokenizer.decode(answer[0], skip_special_tokens=True, skip_prompt=True)
        return prompt, answer

In [6]:
ai_agent = AIAgent(model_path=MODEL_PATH)

`config.hidden_act` is ignored, you should use `config.hidden_activation` instead.
Gemma's activation function will be set to `gelu_pytorch_tanh`. Please, use
`config.hidden_activation` if you want to override this behaviour.
See https://github.com/huggingface/transformers/pull/29402 for more details.


Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

## RAG Pipeline

In [7]:
class RAGSystem:
    """Sentence embedding based Retrieval Based Augmented generation.
        Given database of pdf files, retriever finds num_retrieved_docs relevant documents"""
    def __init__(self, ai_agent, rag_path, num_retrieved_docs=1):
        # load the data
        self.num_docs = num_retrieved_docs
        self.ai_agent = ai_agent
        if '.csv' in rag_path:
            loader = CSVLoader(rag_path)
        else:
            loader = JSONLoader(file_path=rag_path, jq_schema='.documents[].content')
            
        documents = loader.load()
        self.template = "\n\nQuestion:\n{question}\n\nPrompt:\n{prompt}\n\nAnswer:\n{answer}\n\nContext:\n{context}"
        
        text_splitter = RecursiveCharacterTextSplitter(
            chunk_size=800, 
            chunk_overlap=100)
        all_splits = text_splitter.split_documents(documents)
        # create a vectorstore database
        embeddings = HuggingFaceEmbeddings(model_name="sentence-transformers/all-mpnet-base-v2",
                                           model_kwargs = {"device": "cuda"})
        self.vector_db = Chroma.from_documents(documents=all_splits, 
                                               embedding=embeddings, 
                                               persist_directory="chroma_db")
        self.retriever = self.vector_db.as_retriever(search_type="mmr", search_kwargs={'k': self.num_docs})

    def retrieve(self, query):
        # retrieve top k similar documents to query
        docs = self.retriever.get_relevant_documents(query)
        return docs
    
    def query(self, query, video_caption):
        # generate the answer
        context = self.retrieve(query)
        data = ""
        for item in list(context):
            data += item.page_content
            
        data = data[:5000]

        prompt, answer = self.ai_agent.generate(query, video_caption, data)
        
        return self.template.format(question=query,
                                    prompt=prompt,
                                   answer=answer,
                                   context=context)

In [8]:
def colorize_text(text):
    for word, color in zip(["Question", "Prompt", "Answer", "Context"], ["blue", "magenta", "red", "green"]):
        text = text.replace(f"\n\n{word}:", f"\n\n**<font color='{color}'>{word}:</font>**")
    return text

## Test

In [9]:
rag_system = RAGSystem(ai_agent, RAG_PATH, num_retrieved_docs=NUM_RETRIEVED_DOCS)

/tmp/ipykernel_23/2689666795.py:21: LangChainDeprecationWarning: The class `HuggingFaceEmbeddings` was deprecated in LangChain 0.2.2 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-huggingface package and should be used instead. To use it run `pip install -U :class:`~langchain-huggingface` and import as `from :class:`~langchain_huggingface import HuggingFaceEmbeddings``.
  embeddings = HuggingFaceEmbeddings(model_name="sentence-transformers/all-mpnet-base-v2",


modules.json:   0%|          | 0.00/349 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/116 [00:00<?, ?B/s]

README.md:   0%|          | 0.00/10.6k [00:00<?, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/53.0 [00:00<?, ?B/s]

...

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/239 [00:00<?, ?B/s]

/opt/conda/lib/python3.10/site-packages/transformers/tokenization_utils_base.py:1601: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be depracted in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(


1_Pooling/config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

In [10]:
prompt = "Make a script for a commentator's commentary on the {sport} match at the Paris Olympics."
prompt = prompt.format(sport='table tennis', player='between Sin Yubin and Hayata Hina', count=5)
answer = rag_system.query(prompt, VIDEO_CAPTION_1)

display(Markdown(colorize_text(answer)))

/tmp/ipykernel_23/2689666795.py:30: LangChainDeprecationWarning: The method `BaseRetriever.get_relevant_documents` was deprecated in langchain-core 0.1.46 and will be removed in 1.0. Use :meth:`~invoke` instead.
  docs = self.retriever.get_relevant_documents(query)




**<font color='blue'>Question:</font>**
Make a script for a commentator's commentary on the table tennis match at the Paris Olympics.

**<font color='magenta'>Prompt:</font>**

        You are an AI agent specialized in creating commentary script of live sports events.
        Describe the game using the video caption provided (Video Caption).
        Describe the sport and the player using the context provided (Context).
        In order to create the commentary, please use the information from the context provided (Context).
        If needed, include also explanations.
        Video Caption: The video shows a ping pong match between two women playing on a table tennis court. One of the players is wearing a yellow shirt, while the other is wearing a blue shirt. The players are seen hitting the ball back and forth over the net, with the ball landing on the table. The players are actively engaged in the game, displaying their skills and techniques. The video captures the intensity and excitement of the match, showcasing the players' competitive spirit and the fast-paced nature of the sport.
        Question: Make a script for a commentator's commentary on the table tennis match at the Paris Olympics.
        Context: The player in the bronze medal match for table tennis women's singles at the Paris Olympics on August 3, 2024 is Shin Yubin, South Korea's world ranking No 8. player in table tennis women's singles wearing a blue uniform.Badminton is a racket sport whose exact origins are still a tantalising mystery. It evolved from the older games of battledore and shuttlecock, which were popular pastimes in Europe—particularly among the more affluent classes. However, it is unknown exactly when battledore and shuttlecock transformed into the competitive sport of badminton. One plausible theory is that badminton was first played at the stately home of the Duke of Beaufort in Gloucestershire sometime in the early 1860s and was thus named after his estate: Badminton House. The game travelled to India where it became a popular sport in military cantonments, and gradually spread across the British colonies and then to Europe and East Asia. Today, badminton is a global sport with widespread appeal among people of all ages andthe floor exercise, pommel horse, rings, vault, parallel bars, and horizontal bar, while women’s events include the vault, uneven bars, balance beam, and floor exercise. Each element of gymnastic competition requires strength, agility, coordination, and precision. Up until 2004, gymnastic routines at the Games were evaluated with a maximum of 10 points, but from 2005 the mode of scoring changed to a combination of a D score (difficulty/content of the exercise) and an E score (execution) to allow for a greater variation between athletes’ performances. Changes to the scoring system were first considered following the Montreal 1976 Olympic Games, when Romanian gymnast Nadia Comaneci became the first competitor in history to earn a perfect score of 10.0 for her routine on the uneven bars
        Answer:
        

**<font color='red'>Answer:</font>**

        You are an AI agent specialized in creating commentary script of live sports events.
        Describe the game using the video caption provided (Video Caption).
        Describe the sport and the player using the context provided (Context).
        In order to create the commentary, please use the information from the context provided (Context).
        If needed, include also explanations.
        Video Caption: The video shows a ping pong match between two women playing on a table tennis court. One of the players is wearing a yellow shirt, while the other is wearing a blue shirt. The players are seen hitting the ball back and forth over the net, with the ball landing on the table. The players are actively engaged in the game, displaying their skills and techniques. The video captures the intensity and excitement of the match, showcasing the players' competitive spirit and the fast-paced nature of the sport.
        Question: Make a script for a commentator's commentary on the table tennis match at the Paris Olympics.
        Context: The player in the bronze medal match for table tennis women's singles at the Paris Olympics on August 3, 2024 is Shin Yubin, South Korea's world ranking No 8. player in table tennis women's singles wearing a blue uniform.Badminton is a racket sport whose exact origins are still a tantalising mystery. It evolved from the older games of battledore and shuttlecock, which were popular pastimes in Europe—particularly among the more affluent classes. However, it is unknown exactly when battledore and shuttlecock transformed into the competitive sport of badminton. One plausible theory is that badminton was first played at the stately home of the Duke of Beaufort in Gloucestershire sometime in the early 1860s and was thus named after his estate: Badminton House. The game travelled to India where it became a popular sport in military cantonments, and gradually spread across the British colonies and then to Europe and East Asia. Today, badminton is a global sport with widespread appeal among people of all ages andthe floor exercise, pommel horse, rings, vault, parallel bars, and horizontal bar, while women’s events include the vault, uneven bars, balance beam, and floor exercise. Each element of gymnastic competition requires strength, agility, coordination, and precision. Up until 2004, gymnastic routines at the Games were evaluated with a maximum of 10 points, but from 2005 the mode of scoring changed to a combination of a D score (difficulty/content of the exercise) and an E score (execution) to allow for a greater variation between athletes’ performances. Changes to the scoring system were first considered following the Montreal 1976 Olympic Games, when Romanian gymnast Nadia Comaneci became the first competitor in history to earn a perfect score of 10.0 for her routine on the uneven bars
        Answer:
        **Commentator:** Ladies and gentlemen, we are back at the Olympic stadium in Paris, where the battle for bronze medals is heating up. We have a clash of styles in the women's table tennis final, as Shin Yubin, the South Korean powerhouse, takes on her opponent, a determined and skillful opponent.

**Shin:** (With a confident stride) I'm ready to fight for the bronze. My focus is on the game, and I won't be distracted by the crowd.

**Commentator:** The crowd roars as the players take their positions, the tension palpable in the air. The ball flies back and forth, each shot a testament to the skill and precision of these athletes.

**Shin:** (As she hits the ball) It's time to unleash my power!

**Commentator:** The commentators watch as Shin's powerful strokes land, forcing her opponent to react with lightning reflexes. The crowd is on its feet, as the players trade blows with an intensity that is almost palpable.

**Shin:** (With a smile on her face) I'm in the zone, and I'm not giving up.

**Commentator:** The match continues, with both players trading blows with an unwavering determination. The crowd is on its feet, as the players battle it out on the court.

**Shin:** (As she secures the bronze medal) I'm so proud to be standing here, representing my country. I'm going to give it my all!

**Commentator:** And there you have it, ladies and gentlemen, the thrilling conclusion to the women's table tennis final. Shin Yubin, the South Korean powerhouse, has secured the bronze medal, proving that power and determination can overcome any obstacle.

**<font color='green'>Context:</font>**
[Document(metadata={'seq_num': 5, 'source': '/kaggle/input/sport-rag-datafor-sport-commentary/sport_rag_data.json'}, page_content="The player in the bronze medal match for table tennis women's singles at the Paris Olympics on August 3, 2024 is Shin Yubin, South Korea's world ranking No 8. player in table tennis women's singles wearing a blue uniform."), Document(metadata={'seq_num': 2, 'source': '/kaggle/input/sport-rag-datafor-sport-commentary/sport_rag_data.json'}, page_content='Badminton is a racket sport whose exact origins are still a tantalising mystery. It evolved from the older games of battledore and shuttlecock, which were popular pastimes in Europe—particularly among the more affluent classes. However, it is unknown exactly when battledore and shuttlecock transformed into the competitive sport of badminton. One plausible theory is that badminton was first played at the stately home of the Duke of Beaufort in Gloucestershire sometime in the early 1860s and was thus named after his estate: Badminton House. The game travelled to India where it became a popular sport in military cantonments, and gradually spread across the British colonies and then to Europe and East Asia. Today, badminton is a global sport with widespread appeal among people of all ages and'), Document(metadata={'seq_num': 3, 'source': '/kaggle/input/sport-rag-datafor-sport-commentary/sport_rag_data.json'}, page_content='the floor exercise, pommel horse, rings, vault, parallel bars, and horizontal bar, while women’s events include the vault, uneven bars, balance beam, and floor exercise. Each element of gymnastic competition requires strength, agility, coordination, and precision. Up until 2004, gymnastic routines at the Games were evaluated with a maximum of 10 points, but from 2005 the mode of scoring changed to a combination of a D score (difficulty/content of the exercise) and an E score (execution) to allow for a greater variation between athletes’ performances. Changes to the scoring system were first considered following the Montreal 1976 Olympic Games, when Romanian gymnast Nadia Comaneci became the first competitor in history to earn a perfect score of 10.0 for her routine on the uneven bars')]

In [11]:
prompt = "Make a script for a commentator's commentary on the {game} for {sport} {player} at the Paris Olympics based on video caption."
prompt = prompt.format(sport="table tennis women's singles", game = "bronze medal match", player='between Sin Yubin and Hayata Hina', count=5)
answer = rag_system.query(prompt, VIDEO_CAPTION_1)
display(Markdown(colorize_text(answer)))



**<font color='blue'>Question:</font>**
Make a script for a commentator's commentary on the bronze medal match for table tennis women's singles between Sin Yubin and Hayata Hina at the Paris Olympics based on video caption.

**<font color='magenta'>Prompt:</font>**

        You are an AI agent specialized in creating commentary script of live sports events.
        Describe the game using the video caption provided (Video Caption).
        Describe the sport and the player using the context provided (Context).
        In order to create the commentary, please use the information from the context provided (Context).
        If needed, include also explanations.
        Video Caption: The video shows a ping pong match between two women playing on a table tennis court. One of the players is wearing a yellow shirt, while the other is wearing a blue shirt. The players are seen hitting the ball back and forth over the net, with the ball landing on the table. The players are actively engaged in the game, displaying their skills and techniques. The video captures the intensity and excitement of the match, showcasing the players' competitive spirit and the fast-paced nature of the sport.
        Question: Make a script for a commentator's commentary on the bronze medal match for table tennis women's singles between Sin Yubin and Hayata Hina at the Paris Olympics based on video caption.
        Context: The player in the bronze medal match for table tennis women's singles at the Paris Olympics on August 3, 2024 is Hayata Hina, Japan's world ranking No 4. player in table tennis women's singles wearing a blue uniform.Badminton is a racket sport whose exact origins are still a tantalising mystery. It evolved from the older games of battledore and shuttlecock, which were popular pastimes in Europe—particularly among the more affluent classes. However, it is unknown exactly when battledore and shuttlecock transformed into the competitive sport of badminton. One plausible theory is that badminton was first played at the stately home of the Duke of Beaufort in Gloucestershire sometime in the early 1860s and was thus named after his estate: Badminton House. The game travelled to India where it became a popular sport in military cantonments, and gradually spread across the British colonies and then to Europe and East Asia. Today, badminton is a global sport with widespread appeal among people of all ages andthe floor exercise, pommel horse, rings, vault, parallel bars, and horizontal bar, while women’s events include the vault, uneven bars, balance beam, and floor exercise. Each element of gymnastic competition requires strength, agility, coordination, and precision. Up until 2004, gymnastic routines at the Games were evaluated with a maximum of 10 points, but from 2005 the mode of scoring changed to a combination of a D score (difficulty/content of the exercise) and an E score (execution) to allow for a greater variation between athletes’ performances. Changes to the scoring system were first considered following the Montreal 1976 Olympic Games, when Romanian gymnast Nadia Comaneci became the first competitor in history to earn a perfect score of 10.0 for her routine on the uneven bars
        Answer:
        

**<font color='red'>Answer:</font>**

        You are an AI agent specialized in creating commentary script of live sports events.
        Describe the game using the video caption provided (Video Caption).
        Describe the sport and the player using the context provided (Context).
        In order to create the commentary, please use the information from the context provided (Context).
        If needed, include also explanations.
        Video Caption: The video shows a ping pong match between two women playing on a table tennis court. One of the players is wearing a yellow shirt, while the other is wearing a blue shirt. The players are seen hitting the ball back and forth over the net, with the ball landing on the table. The players are actively engaged in the game, displaying their skills and techniques. The video captures the intensity and excitement of the match, showcasing the players' competitive spirit and the fast-paced nature of the sport.
        Question: Make a script for a commentator's commentary on the bronze medal match for table tennis women's singles between Sin Yubin and Hayata Hina at the Paris Olympics based on video caption.
        Context: The player in the bronze medal match for table tennis women's singles at the Paris Olympics on August 3, 2024 is Hayata Hina, Japan's world ranking No 4. player in table tennis women's singles wearing a blue uniform.Badminton is a racket sport whose exact origins are still a tantalising mystery. It evolved from the older games of battledore and shuttlecock, which were popular pastimes in Europe—particularly among the more affluent classes. However, it is unknown exactly when battledore and shuttlecock transformed into the competitive sport of badminton. One plausible theory is that badminton was first played at the stately home of the Duke of Beaufort in Gloucestershire sometime in the early 1860s and was thus named after his estate: Badminton House. The game travelled to India where it became a popular sport in military cantonments, and gradually spread across the British colonies and then to Europe and East Asia. Today, badminton is a global sport with widespread appeal among people of all ages andthe floor exercise, pommel horse, rings, vault, parallel bars, and horizontal bar, while women’s events include the vault, uneven bars, balance beam, and floor exercise. Each element of gymnastic competition requires strength, agility, coordination, and precision. Up until 2004, gymnastic routines at the Games were evaluated with a maximum of 10 points, but from 2005 the mode of scoring changed to a combination of a D score (difficulty/content of the exercise) and an E score (execution) to allow for a greater variation between athletes’ performances. Changes to the scoring system were first considered following the Montreal 1976 Olympic Games, when Romanian gymnast Nadia Comaneci became the first competitor in history to earn a perfect score of 10.0 for her routine on the uneven bars
        Answer:
        **Welcome to the bronze medal match for the table tennis women's singles at the Paris Olympics!**

**Let's take a look at the players, shall we? Hayata Hina, sporting the blue uniform, is the reigning world champion, ranked number four in the world. Sin Yubin, representing Japan, is looking to make a statement here.

**The crowd is roaring as the players take their positions on the court, ready to battle it out!**

**The ball is in play, and Hayata and Sin are trading shots with lightning speed!**

**Hayata's powerful serve sends the ball soaring into the net, but Sin is ready to receive!**

**The rallies are intense, back and forth, with both players showcasing their incredible skills!**

**The crowd is on its feet, as the players battle for every point!**

**With seconds left on the clock, Hayata finds herself with the opportunity to serve again. She goes for the gold, but Sin is ready with a powerful block!**

**The crowd goes wild as the referee makes the call! Hayata Hina has won the bronze medal!**

**What a thrilling match! We've witnessed some incredible skills and determination from both players. Congratulations to Hayata Hina for securing the bronze medal!**

**<font color='green'>Context:</font>**
[Document(metadata={'seq_num': 6, 'source': '/kaggle/input/sport-rag-datafor-sport-commentary/sport_rag_data.json'}, page_content="The player in the bronze medal match for table tennis women's singles at the Paris Olympics on August 3, 2024 is Hayata Hina, Japan's world ranking No 4. player in table tennis women's singles wearing a blue uniform."), Document(metadata={'seq_num': 2, 'source': '/kaggle/input/sport-rag-datafor-sport-commentary/sport_rag_data.json'}, page_content='Badminton is a racket sport whose exact origins are still a tantalising mystery. It evolved from the older games of battledore and shuttlecock, which were popular pastimes in Europe—particularly among the more affluent classes. However, it is unknown exactly when battledore and shuttlecock transformed into the competitive sport of badminton. One plausible theory is that badminton was first played at the stately home of the Duke of Beaufort in Gloucestershire sometime in the early 1860s and was thus named after his estate: Badminton House. The game travelled to India where it became a popular sport in military cantonments, and gradually spread across the British colonies and then to Europe and East Asia. Today, badminton is a global sport with widespread appeal among people of all ages and'), Document(metadata={'seq_num': 3, 'source': '/kaggle/input/sport-rag-datafor-sport-commentary/sport_rag_data.json'}, page_content='the floor exercise, pommel horse, rings, vault, parallel bars, and horizontal bar, while women’s events include the vault, uneven bars, balance beam, and floor exercise. Each element of gymnastic competition requires strength, agility, coordination, and precision. Up until 2004, gymnastic routines at the Games were evaluated with a maximum of 10 points, but from 2005 the mode of scoring changed to a combination of a D score (difficulty/content of the exercise) and an E score (execution) to allow for a greater variation between athletes’ performances. Changes to the scoring system were first considered following the Montreal 1976 Olympic Games, when Romanian gymnast Nadia Comaneci became the first competitor in history to earn a perfect score of 10.0 for her routine on the uneven bars')]

In [12]:
prompt = "Make a script for a commentator's commentary on the {game} for {sport} {player} at the Paris Olympics based on video caption."
prompt = prompt.format(sport='artistic gymnastics women’s vault', game = "final", player='Simon Biles', count=5)
answer = rag_system.query(prompt, VIDEO_CAPTION_2)
display(Markdown(colorize_text(answer)))



**<font color='blue'>Question:</font>**
Make a script for a commentator's commentary on the final for artistic gymnastics women’s vault Simon Biles at the Paris Olympics based on video caption.

**<font color='magenta'>Prompt:</font>**

        You are an AI agent specialized in creating commentary script of live sports events.
        Describe the game using the video caption provided (Video Caption).
        Describe the sport and the player using the context provided (Context).
        In order to create the commentary, please use the information from the context provided (Context).
        If needed, include also explanations.
        Video Caption: The video features a female gymnast performing a routine on a balance beam. She starts by jumping onto the beam and executing a series of flips and turns, showcasing her impressive athleticism and balance. The gymnast then jumps off the beam and lands on the mat with her arms raised in the air, indicating a successful performance. The crowd cheers and applauds her as she completes the routine.
        Question: Make a script for a commentator's commentary on the final for artistic gymnastics women’s vault Simon Biles at the Paris Olympics based on video caption.
        Context: The player of the final for the artistic gymnastics women’s vault at the Paris Olympics on August 3, 2024 is Simone Biles, USA's one of the greatest gymnasts of all time wearing a red uniform.the first competitor in history to earn a perfect score of 10.0 for her routine on the uneven bars during the team competition. Artistic gymnastics was introduced at the very first Olympic Games of the modern era in 1896 and has been included in every edition since. The competition was restricted to male competitors for 32 years until the 1928 Olympic Games in Amsterdam, when women were allowed to compete for the first time. It wasn't until 1952 that the women’s programme was developed with seven events, and then later stabilised at six events, as has been the case since the 1960 Games in Rome. There are eight events on the men’s programme.Badminton is a racket sport whose exact origins are still a tantalising mystery. It evolved from the older games of battledore and shuttlecock, which were popular pastimes in Europe—particularly among the more affluent classes. However, it is unknown exactly when battledore and shuttlecock transformed into the competitive sport of badminton. One plausible theory is that badminton was first played at the stately home of the Duke of Beaufort in Gloucestershire sometime in the early 1860s and was thus named after his estate: Badminton House. The game travelled to India where it became a popular sport in military cantonments, and gradually spread across the British colonies and then to Europe and East Asia. Today, badminton is a global sport with widespread appeal among people of all ages and
        Answer:
        

**<font color='red'>Answer:</font>**

        You are an AI agent specialized in creating commentary script of live sports events.
        Describe the game using the video caption provided (Video Caption).
        Describe the sport and the player using the context provided (Context).
        In order to create the commentary, please use the information from the context provided (Context).
        If needed, include also explanations.
        Video Caption: The video features a female gymnast performing a routine on a balance beam. She starts by jumping onto the beam and executing a series of flips and turns, showcasing her impressive athleticism and balance. The gymnast then jumps off the beam and lands on the mat with her arms raised in the air, indicating a successful performance. The crowd cheers and applauds her as she completes the routine.
        Question: Make a script for a commentator's commentary on the final for artistic gymnastics women’s vault Simon Biles at the Paris Olympics based on video caption.
        Context: The player of the final for the artistic gymnastics women’s vault at the Paris Olympics on August 3, 2024 is Simone Biles, USA's one of the greatest gymnasts of all time wearing a red uniform.the first competitor in history to earn a perfect score of 10.0 for her routine on the uneven bars during the team competition. Artistic gymnastics was introduced at the very first Olympic Games of the modern era in 1896 and has been included in every edition since. The competition was restricted to male competitors for 32 years until the 1928 Olympic Games in Amsterdam, when women were allowed to compete for the first time. It wasn't until 1952 that the women’s programme was developed with seven events, and then later stabilised at six events, as has been the case since the 1960 Games in Rome. There are eight events on the men’s programme.Badminton is a racket sport whose exact origins are still a tantalising mystery. It evolved from the older games of battledore and shuttlecock, which were popular pastimes in Europe—particularly among the more affluent classes. However, it is unknown exactly when battledore and shuttlecock transformed into the competitive sport of badminton. One plausible theory is that badminton was first played at the stately home of the Duke of Beaufort in Gloucestershire sometime in the early 1860s and was thus named after his estate: Badminton House. The game travelled to India where it became a popular sport in military cantonments, and gradually spread across the British colonies and then to Europe and East Asia. Today, badminton is a global sport with widespread appeal among people of all ages and
        Answer:
        **Commentator Script:**

"Ladies and gentlemen, we are back in Paris for the final of artistic gymnastics, and the stage is set for Simone Biles, the undisputed queen of the vault. She's been warming up with a series of impressive flips and turns, showcasing her incredible athleticism and balance.

With a perfect score of 10.0, she's the first competitor in history to earn a perfect score on the uneven bars during the team competition. Her performance is nothing short of breathtaking, and the crowd is roaring in appreciation.

As she takes her final jump, she raises her arms in the air, indicating her hard-earned victory. The crowd erupts in thunderous applause, recognizing her exceptional performance.

This is a moment that will forever be etched in the annals of Olympic history, as Simone Biles emerges as the undisputed queen of artistic gymnastics."

**<font color='green'>Context:</font>**
[Document(metadata={'seq_num': 9, 'source': '/kaggle/input/sport-rag-datafor-sport-commentary/sport_rag_data.json'}, page_content="The player of the final for the artistic gymnastics women’s vault at the Paris Olympics on August 3, 2024 is Simone Biles, USA's one of the greatest gymnasts of all time wearing a red uniform."), Document(metadata={'seq_num': 3, 'source': '/kaggle/input/sport-rag-datafor-sport-commentary/sport_rag_data.json'}, page_content="the first competitor in history to earn a perfect score of 10.0 for her routine on the uneven bars during the team competition. Artistic gymnastics was introduced at the very first Olympic Games of the modern era in 1896 and has been included in every edition since. The competition was restricted to male competitors for 32 years until the 1928 Olympic Games in Amsterdam, when women were allowed to compete for the first time. It wasn't until 1952 that the women’s programme was developed with seven events, and then later stabilised at six events, as has been the case since the 1960 Games in Rome. There are eight events on the men’s programme."), Document(metadata={'seq_num': 2, 'source': '/kaggle/input/sport-rag-datafor-sport-commentary/sport_rag_data.json'}, page_content='Badminton is a racket sport whose exact origins are still a tantalising mystery. It evolved from the older games of battledore and shuttlecock, which were popular pastimes in Europe—particularly among the more affluent classes. However, it is unknown exactly when battledore and shuttlecock transformed into the competitive sport of badminton. One plausible theory is that badminton was first played at the stately home of the Duke of Beaufort in Gloucestershire sometime in the early 1860s and was thus named after his estate: Badminton House. The game travelled to India where it became a popular sport in military cantonments, and gradually spread across the British colonies and then to Europe and East Asia. Today, badminton is a global sport with widespread appeal among people of all ages and')]

In [13]:
prompt = "Make a script for a commentator's commentary on the {game} for {sport} {player} at the Paris Olympics based on video caption."
prompt = prompt.format(sport='diving', game = "final for the diving men's 10m platform", player='Chao Yuan', count=5)
answer = rag_system.query(prompt, VIDEO_CAPTION_3)
display(Markdown(colorize_text(answer)))



**<font color='blue'>Question:</font>**
Make a script for a commentator's commentary on the final for the diving men's 10m platform for diving Chao Yuan at the Paris Olympics based on video caption.

**<font color='magenta'>Prompt:</font>**

        You are an AI agent specialized in creating commentary script of live sports events.
        Describe the game using the video caption provided (Video Caption).
        Describe the sport and the player using the context provided (Context).
        In order to create the commentary, please use the information from the context provided (Context).
        If needed, include also explanations.
        Video Caption: In the video, a man is seen diving into a pool from a diving board, and he is captured in mid-air as he jumps off the board. The man's body is seen arching as he descends into the water, creating a splash. The dive is executed with precision and skill, showcasing the diver's athleticism and control. The video captures the moment of impact as the man hits the water, creating a visually striking scene. The dive is a testament to the diver's abilities and the beauty of the sport.
        Question: Make a script for a commentator's commentary on the final for the diving men's 10m platform for diving Chao Yuan at the Paris Olympics based on video caption.
        Context: The player of the final for the diving men's 10m platform at the Paris Olympics on August 10, 2024 is Chao Yuan, China's one of the greatest gymnasts of all time wearing a yellow uniformair, while the high dive is performed from a fixed platform position 10 metres above the water. The individual and synchronised competitions take place at both heights. A panel of judges score each dive based on various criteria, including how aesthetically pleasing a diver’s movements are, the complexity of the dive and how well the diver enters the water. Synchronised diving is also scored on how well the two divers match each other’s movements. Diving made its Olympic debut at the 1904 Games in St. Louis and has been included at every Games since. The first women’s events were contested at the 1912 Games in Stockholm, while the synchronised competition was added to the programme at the Sydney 2000 Olympic Games. Diving events at the Games were initially dominated by Team USA athletes,East Asia. Today, badminton is a global sport with widespread appeal among people of all ages and abilities. Badminton is a racket sport played indoors by two opposing players (singles) or two pairs (doubles), who take positions on opposite halves of a court divided by a net. Players use a shuttlecock (a projectile with a crown of feathers in a conical shape) during a rally, with the aim of trying to land it on the surface of their opponents' court or forcing their opponents to make an error. A point is won/lost every serve in the game. Players must win two games of 21 points to win the match (best of three). Badminton made its Olympic debut as a demonstration sport at the 1972 Games in Munich, and was included as an exhibition sport at the Seoul 1988 Olympic Games; four years later
        Answer:
        

**<font color='red'>Answer:</font>**

        You are an AI agent specialized in creating commentary script of live sports events.
        Describe the game using the video caption provided (Video Caption).
        Describe the sport and the player using the context provided (Context).
        In order to create the commentary, please use the information from the context provided (Context).
        If needed, include also explanations.
        Video Caption: In the video, a man is seen diving into a pool from a diving board, and he is captured in mid-air as he jumps off the board. The man's body is seen arching as he descends into the water, creating a splash. The dive is executed with precision and skill, showcasing the diver's athleticism and control. The video captures the moment of impact as the man hits the water, creating a visually striking scene. The dive is a testament to the diver's abilities and the beauty of the sport.
        Question: Make a script for a commentator's commentary on the final for the diving men's 10m platform for diving Chao Yuan at the Paris Olympics based on video caption.
        Context: The player of the final for the diving men's 10m platform at the Paris Olympics on August 10, 2024 is Chao Yuan, China's one of the greatest gymnasts of all time wearing a yellow uniformair, while the high dive is performed from a fixed platform position 10 metres above the water. The individual and synchronised competitions take place at both heights. A panel of judges score each dive based on various criteria, including how aesthetically pleasing a diver’s movements are, the complexity of the dive and how well the diver enters the water. Synchronised diving is also scored on how well the two divers match each other’s movements. Diving made its Olympic debut at the 1904 Games in St. Louis and has been included at every Games since. The first women’s events were contested at the 1912 Games in Stockholm, while the synchronised competition was added to the programme at the Sydney 2000 Olympic Games. Diving events at the Games were initially dominated by Team USA athletes,East Asia. Today, badminton is a global sport with widespread appeal among people of all ages and abilities. Badminton is a racket sport played indoors by two opposing players (singles) or two pairs (doubles), who take positions on opposite halves of a court divided by a net. Players use a shuttlecock (a projectile with a crown of feathers in a conical shape) during a rally, with the aim of trying to land it on the surface of their opponents' court or forcing their opponents to make an error. A point is won/lost every serve in the game. Players must win two games of 21 points to win the match (best of three). Badminton made its Olympic debut as a demonstration sport at the 1972 Games in Munich, and was included as an exhibition sport at the Seoul 1988 Olympic Games; four years later
        Answer:
        **Intro**
"Good evening, sports fans, and welcome to the final of the diving men's 10m platform at the Paris Olympics. We have a battle for gold between two of the greatest gymnasts of all time, Chao Yuan from China and the defending champion, [Name of the competitor from another country].

**The Dive**
"The athletes take their positions on the platform, and the crowd is on its feet as the music starts. Chao Yuan jumps into the air with incredible precision and grace, his body arching as he prepares to make the dive. He executes a perfect somersault dive, showcasing his incredible athleticism and control. The crowd is on its feet, and the judges are watching the performance with rapt attention."

**The Aftermath**
"The impact is visually stunning as Chao Yuan hits the water with a splash, creating a moment of pure exhilaration. His body is perfectly arched, and his performance is a testament to the beauty of the sport. The judges give him a standing ovation for his incredible performance."

**Conclusion**
"And that concludes the final of the diving men's 10m platform at the Paris Olympics. Chao Yuan has secured his gold medal, and the crowd erupts in applause. It's been an incredible final, and we've witnessed some of the best diving the world has ever seen. Thank you for joining us for the coverage of the Paris Olympics."

**<font color='green'>Context:</font>**
[Document(metadata={'seq_num': 10, 'source': '/kaggle/input/sport-rag-datafor-sport-commentary/sport_rag_data.json'}, page_content="The player of the final for the diving men's 10m platform at the Paris Olympics on August 10, 2024 is Chao Yuan, China's one of the greatest gymnasts of all time wearing a yellow uniform"), Document(metadata={'seq_num': 4, 'source': '/kaggle/input/sport-rag-datafor-sport-commentary/sport_rag_data.json'}, page_content='air, while the high dive is performed from a fixed platform position 10 metres above the water. The individual and synchronised competitions take place at both heights. A panel of judges score each dive based on various criteria, including how aesthetically pleasing a diver’s movements are, the complexity of the dive and how well the diver enters the water. Synchronised diving is also scored on how well the two divers match each other’s movements. Diving made its Olympic debut at the 1904 Games in St. Louis and has been included at every Games since. The first women’s events were contested at the 1912 Games in Stockholm, while the synchronised competition was added to the programme at the Sydney 2000 Olympic Games. Diving events at the Games were initially dominated by Team USA athletes,'), Document(metadata={'seq_num': 2, 'source': '/kaggle/input/sport-rag-datafor-sport-commentary/sport_rag_data.json'}, page_content="East Asia. Today, badminton is a global sport with widespread appeal among people of all ages and abilities. Badminton is a racket sport played indoors by two opposing players (singles) or two pairs (doubles), who take positions on opposite halves of a court divided by a net. Players use a shuttlecock (a projectile with a crown of feathers in a conical shape) during a rally, with the aim of trying to land it on the surface of their opponents' court or forcing their opponents to make an error. A point is won/lost every serve in the game. Players must win two games of 21 points to win the match (best of three). Badminton made its Olympic debut as a demonstration sport at the 1972 Games in Munich, and was included as an exhibition sport at the Seoul 1988 Olympic Games; four years later")]

## Reference

- [RAG using Gemma, Langchain and ChromaDB by GABRIEL PREDA](https://www.kaggle.com/code/gpreda/rag-using-gemma-langchain-and-chromadb)
- [RAG Using Langchain, ChromaDB, Ollama and Gemma 7b by DEEPAK GUPTA](https://www.kaggle.com/code/deeepsig/rag-using-langchain-chromadb-ollama-and-gemma-7b)
- [Advanced RAG with Gemma, Weaviate, and LlamaIndex by LEONIE](https://www.kaggle.com/code/iamleonie/advanced-rag-with-gemma-weaviate-and-llamaindex#Step-3:-Load-data)